<a href="https://colab.research.google.com/github/Bhavnicksm/ABSA-SentiHood/blob/main/SentiHood_BERT_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece
!pip install datasets

     |████████████████████████████████| 3.4 MB 4.4 MB/s 
     |████████████████████████████████| 1.2 MB 37.5 MB/s 
     |████████████████████████████████| 895 kB 38.0 MB/s 
     |████████████████████████████████| 61 kB 490 kB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
     |████████████████████████████████| 596 kB 50.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 306 kB 4.2 MB/s 
     |████████████████████████████████| 243 kB 45.2 MB/s 
     |████████████████████████████████| 1.1 MB 40.8 MB/s 
     |████████████████████████████████| 132 kB 50.7 MB/s 
     |████████████████████████████████| 160 kB 51.0 MB/s 
     |████████████████████████████████| 192 kB 45.9 MB/s 
     |████████████████████████████████| 271 kB 50.1 MB/s 


# Loading the Libraries & Models

In [2]:
import json
from collections import Counter
from typing import Any, Sequence, Optional, Union

from pprint import pprint
from tqdm.notebook import tqdm, trange

import numpy as np


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset, DataLoader

from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TrainingArguments,
                          Trainer)
import datasets

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

Device: cuda


In [3]:
tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased")
model = AutoModelForSequenceClassification.from_pretrained('bert-large-uncased', num_labels=3).to(device)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

# Preprocessing

In [4]:
train_path = "/content/drive/MyDrive/SentiHood/data/sentihood-train.json"

with open(train_path, 'r') as file:
  train_dict = json.loads(file.read())

pprint(train_dict[22])

{'id': 61,
 'opinions': [{'aspect': 'live',
               'sentiment': 'Positive',
               'target_entity': 'LOCATION1'}],
 'text': '  I really like LOCATION1  I lived there for 3 months and had a lot '
         'of fun'}


In [5]:
dev_path = "/content/drive/MyDrive/SentiHood/data/sentihood-dev.json"

with open(dev_path, 'r') as file:
  dev_dict = json.loads(file.read())

pprint(dev_dict[22])

{'id': 593,
 'opinions': [],
 'text': 'Above all the fact that London London and London town were two '
         'different jurisdictions, and LOCATION1 had its own jurisdiction'}


In [6]:
test_path = "/content/drive/MyDrive/SentiHood/data/sentihood-test.json"

with open(test_path, 'r') as file:
  test_dict = json.loads(file.read())

pprint(test_dict[22])

{'id': 1311,
 'opinions': [{'aspect': 'live',
               'sentiment': 'Positive',
               'target_entity': 'LOCATION2'},
              {'aspect': 'live',
               'sentiment': 'Positive',
               'target_entity': 'LOCATION1'}],
 'text': "  I'd say live in the LOCATION1, particularly in the LOCATION2 area"}


In [7]:
# class SentiHoodDataset (Dataset):
  
#   def __init__(self,
#                dataset_dict: Sequence[dict],
#                tokenizer: AutoTokenizer,
#                ):
#     super(SentiHoodDataset, self).__init__()

#     aspect_counter = Counter()
#     for i in range(len(dataset_dict)):
#         for j in range(len(dataset_dict[i]['opinions'])):
#           aspect_counter.update([dataset_dict[i]['opinions'][j]['aspect']])

#     self.aspect_to_id =  {a:i for i,(a,b) in enumerate(aspect_counter.most_common())}
#     # pprint(aspect_to_id)

#     te_counter = Counter()
#     for i in range(len(dataset_dict)):
#       for j in range(len(dataset_dict[i]['opinions'])):
#         te_counter.update([dataset_dict[i]['opinions'][j]['target_entity']])

#     self.te_to_id =  {a : i+1 for i,(a,b) in enumerate(te_counter.most_common())}
#     # pprint(te_to_id)

#     self.senti_to_id = {'None' : 0,
#                         'Positive' : 2,
#                         'Negative' : 1
#                         }

#     binary_targets = set()
#     for te in self.te_to_id.keys():
#       for aspect in self.aspect_to_id.keys():
#         for sentiment in self.senti_to_id.keys():
#           binary_targets.add((te, aspect, sentiment))

#     self.sentence_pairs = []
#     self.labels = []
#     for example in dataset_dict:
#       sent_a = example['text']

#       opinion_set = set()

#       for opinion in example['opinions']:
        
#         te = opinion['target_entity']
#         asp = opinion['aspect']
#         sen = opinion['sentiment']
        
#         loc = 'location - ' + str(self.te_to_id[te])
        
#         sent_b = loc + ' - ' + asp + ' - ' + sen
#         label = 1

#         self.sentence_pairs.append((sent_a, sent_b))
#         self.labels.append(torch.tensor(label))
#         opinion_set.add((te, asp, sen))

#       dummy_index = np.arange(len(binary_targets.difference(opinion_set)))
#       # print(dummy_index)
#       neg_indices = list(np.random.choice(dummy_index, len(opinion_set), replace=False))
#       # print(neg_indices)
#       neg_samples = np.asarray(list(binary_targets.difference(opinion_set)))[neg_indices]
#       for (te, asp, sen) in neg_samples:
#         loc = 'location - ' + str(self.te_to_id[te])
        
#         sent_b = loc + ' - ' + asp + ' - ' + sen
#         label = 0

#         self.sentence_pairs.append((sent_a, sent_b))
#         self.labels.append(torch.tensor(label))

#     self.labels = torch.tensor(self.labels)

#     a = [text_pair[0] for text_pair in self.sentence_pairs]
#     b = [text_pair[1] for text_pair in self.sentence_pairs]
#     self.tokenizer_output = tokenizer(a, b,
#                               padding=True,
#                               truncation = 'only_first',
#                               return_tensors='pt',
#                               return_token_type_ids=True,
#                               return_attention_mask=True,
#                               )
  
#     self.tokenizer_output['labels'] = self.labels
    
    
#   def __len__(self):
#     return len(self.tokenizer_output.shape[0])
  
#   def get_dataset(self):
#     return self.tokenizer_output
  
#   # def get_dataloader(self, batch_size: int):
#   #   return DataLoader(self.dataset, batch_size = batch_size, shuffle=True)

In [8]:
class SentiHoodDataset (Dataset):
  
  def __init__(self,
               dataset_dict: Sequence[dict],
               tokenizer: AutoTokenizer,
               ):
    super(SentiHoodDataset, self).__init__()

    aspect_counter = Counter()
    for i in range(len(dataset_dict)):
        for j in range(len(dataset_dict[i]['opinions'])):
          aspect_counter.update([dataset_dict[i]['opinions'][j]['aspect']])

    self.aspect_to_id =  {a:i for i,(a,b) in enumerate(aspect_counter.most_common())}
    # pprint(aspect_to_id)

    te_counter = Counter()
    for i in range(len(dataset_dict)):
      for j in range(len(dataset_dict[i]['opinions'])):
        te_counter.update([dataset_dict[i]['opinions'][j]['target_entity']])

    self.te_to_id =  {a : i+1 for i,(a,b) in enumerate(te_counter.most_common())}
    # pprint(te_to_id)

    self.senti_to_id = {'None' : 0,
                        'Positive' : 2,
                        'Negative' : 1
                        }

    binary_targets = set()
    for te in self.te_to_id.keys():
      for aspect in self.aspect_to_id.keys():
        binary_targets.add((te, aspect))

    self.sentence_pairs = []
    self.labels = []
    for example in dataset_dict:
      sent_a = example['text']

      opinion_set = set()

      for opinion in example['opinions']:
        
        te = opinion['target_entity']
        asp = opinion['aspect']
        sen = opinion['sentiment']
        
        loc = 'location - ' + str(self.te_to_id[te])
        
        sent_b = loc + ' - ' + asp 
        label = self.senti_to_id[sen]

        self.sentence_pairs.append((sent_a, sent_b))
        self.labels.append(torch.tensor(label))
        opinion_set.add((te, asp))

      dummy_index = np.arange(len(binary_targets.difference(opinion_set)))
      # print(dummy_index)
      neg_indices = list(np.random.choice(dummy_index, len(dummy_index), replace=False))
      # print(neg_indices)
      neg_samples = np.asarray(list(binary_targets.difference(opinion_set)))[neg_indices]
      for (te, asp) in neg_samples:
        loc = 'location - ' + str(self.te_to_id[te])
        
        sent_b = loc + ' - ' + asp
        sen = 'None' 
        label = self.senti_to_id[sen]

        self.sentence_pairs.append((sent_a, sent_b))
        self.labels.append(torch.tensor(label))

    self.labels = torch.tensor(self.labels)

    a = [text_pair[0] for text_pair in self.sentence_pairs]
    b = [text_pair[1] for text_pair in self.sentence_pairs]
    self.tokenizer_output = tokenizer(a, b,
                              padding=True,
                              truncation = 'only_first',
                              return_tensors='pt',
                              return_token_type_ids=True,
                              return_attention_mask=True,
                              )
  
    self.tokenizer_output['labels'] = self.labels
    
    
  def __len__(self):
    return len(self.tokenizer_output.shape[0])
  
  def get_dataset(self):
    return self.tokenizer_output
  
  # def get_dataloader(self, batch_size: int):
  #   return DataLoader(self.dataset, batch_size = batch_size, shuffle=True)

In [9]:
train_ds = SentiHoodDataset(train_dict, tokenizer)
dev_ds = SentiHoodDataset(dev_dict, tokenizer)
test_ds = SentiHoodDataset(test_dict, tokenizer)

In [10]:
train_dataset = datasets.Dataset.from_dict(train_ds.get_dataset())
dev_dataset = datasets.Dataset.from_dict(dev_ds.get_dataset())
test_dataset = datasets.Dataset.from_dict(test_ds.get_dataset())

## Train Loop

In [11]:
import numpy as np
from datasets import load_metric

accuracy = load_metric("accuracy")
recall = load_metric("recall")
precision = load_metric("precision")
f1 = load_metric("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    rec = recall.compute(predictions=predictions, references=labels, average='macro')["recall"]
    pre = precision.compute(predictions=predictions, references=labels, average='macro')["precision"]
    f = f1.compute(predictions=predictions, references=labels, average='macro')["f1"]


    return {"accuracy" : acc,
            "recall"   : rec,
            "precision": pre,
            "f1"       : f,
            }

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.07k [00:00<?, ?B/s]

In [12]:
training_args = TrainingArguments(output_dir = "/content/drive/MyDrive/SentiHood/models/bert-large-uncased",
                                  overwrite_output_dir = True,
                                  resume_from_checkpoint="/content/drive/MyDrive/SentiHood/models/bert-large-uncased",
                                  num_train_epochs = 5,
                                  evaluation_strategy="steps",
                                  eval_steps=1000,
                                  logging_steps=1000, 
                                  per_device_train_batch_size = 8,
                                  per_device_eval_batch_size = 8,
                                  learning_rate = 5e-6,
                                  warmup_ratio = 0.1,
                                  weight_decay = 0.01,
                                  save_strategy = 'epoch',
                                  seed = 42,
                                  )

In [13]:
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=dev_dataset,
                  compute_metrics=compute_metrics, 
                  )

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 71449
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 44660


Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1000,0.681200,0.270765,0.953257,0.333333,0.317752,0.325356
2000,0.249000,0.228778,0.953257,0.333333,0.317752,0.325356


***** Running Evaluation *****
  Num examples = 17928
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17928
  Batch size = 8
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
***** Running Evaluation *****
  Num examples = 17928
  Batch size = 8


Step,Training Loss,Validation Loss,Accuracy,Recall,Precision,F1
1000,0.681200,0.270765,0.953257,0.333333,0.317752,0.325356
2000,0.249000,0.228778,0.953257,0.333333,0.317752,0.325356


In [ ]:
 trainer.evaluate()

In [ ]:
test_output = trainer.predict(test_dataset, )
print(test_output)

In [ ]:
torch.save(test_output, "/content/drive/MyDrive/SentiHood/models/bert-base-uncased/test_output")

In [ ]:
torch.load("/content/drive/MyDrive/SentiHood/models/bert-base-uncased/test_output")

In [ ]:
#preparing the final outputs

output_predictions_list = [(text, nli, pred) for (text, nli), pred in zip(test_ds.sentence_pairs, test_output.label_ids)]

# pprint(output_predictions_list)

result = []

id_to_sent = ['None', 'Negative', 'Positive']


for example in test_dict:
  # pprint(example)
  a = [text for text in output_predictions_list if text[0] is example['text']]
  # print('\n')
  # pprint(a)
  
  model_predictions = []
  for (premise, hyp, sent) in a:

    # print(sent)
    if sent != 0:
      _ , num, asp = hyp.split(' - ')
      loc = 'LOCATION'+str(num)
      # print(loc, num, asp)
      pred_opinion = {'aspect': asp,
                      'sentiment':id_to_sent[sent],
                      'target_entity': loc
                      }
      model_predictions.append(pred_opinion)
  # pprint(model_predictions)
  example['model_pred'] = model_predictions
  # pprint(example)
  
  result.append(example)

print(len(test_dict) , len(result))

In [ ]:
save_path = '/content/drive/MyDrive/SentiHood/models/bert-base-uncased/submission.jsonl'
with open(save_path, 'w') as file:
  file.write(json.dumps(result, indent=4))

# Make eval script

In [ ]:
# class TestDataset (Dataset):
  
#   def __init__(self,
#                dataset_dict: Sequence[dict],
#                tokenizer: AutoTokenizer,
#                ):
#     super(SentiHoodDataset, self).__init__()

#     aspect_counter = Counter()
#     for i in range(len(dataset_dict)):
#         for j in range(len(dataset_dict[i]['opinions'])):
#           aspect_counter.update([dataset_dict[i]['opinions'][j]['aspect']])

#     self.aspect_to_id =  {a:i for i,(a,b) in enumerate(aspect_counter.most_common())}
#     # pprint(aspect_to_id)

#     te_counter = Counter()
#     for i in range(len(dataset_dict)):
#       for j in range(len(dataset_dict[i]['opinions'])):
#         te_counter.update([dataset_dict[i]['opinions'][j]['target_entity']])

#     self.te_to_id =  {a : i+1 for i,(a,b) in enumerate(te_counter.most_common())}
#     # pprint(te_to_id)

#     self.senti_to_id = {'None' : 0,
#                         'Positive' : 2,
#                         'Negative' : 1
#                         }

#     binary_targets = set()
#     for te in self.te_to_id.keys():
#       for aspect in self.aspect_to_id.keys():
#         for sentiment in self.senti_to_id.keys():
#           binary_targets.add((te, aspect, sentiment))

#     self.sentence_pairs = []
#     self.labels = []
#     for example in dataset_dict:
#       sent_a = example['text']

#       opinion_set = set()

#       for opinion in example['opinions']:
        
#         te = opinion['target_entity']
#         asp = opinion['aspect']
#         sen = opinion['sentiment']
        
#         loc = 'location - ' + str(self.te_to_id[te])
        
#         sent_b = loc + ' - ' + asp + ' - ' + sen
#         label = 1

#         self.sentence_pairs.append((sent_a, sent_b))
#         self.labels.append(torch.tensor(label))
#         opinion_set.add((te, asp, sen))

#       dummy_index = np.arange(len(binary_targets.difference(opinion_set)))
#       # print(dummy_index)
#       neg_indices = list(np.random.choice(dummy_index, len(dummy_index), replace=False))
#       # print(neg_indices)
#       neg_samples = np.asarray(list(binary_targets.difference(opinion_set)))[neg_indices]
#       for (te, asp, sen) in neg_samples:
#         loc = 'location - ' + str(self.te_to_id[te])
        
#         sent_b = loc + ' - ' + asp + ' - ' + sen
#         label = 0

#         self.sentence_pairs.append((sent_a, sent_b))
#         self.labels.append(torch.tensor(label))

#     self.labels = torch.tensor(self.labels)

#     a = [text_pair[0] for text_pair in self.sentence_pairs]
#     b = [text_pair[1] for text_pair in self.sentence_pairs]
#     self.tokenizer_output = tokenizer(a, b,
#                               padding=True,
#                               truncation = 'only_first',
#                               return_tensors='pt',
#                               return_token_type_ids=True,
#                               return_attention_mask=True,
#                               )
  
#     self.tokenizer_output['labels'] = self.labels
    
    
#   def __len__(self):
#     return len(self.tokenizer_output.shape[0])
  
#   def get_dataset(self):
#     return self.tokenizer_output
  
#   # def get_dataloader(self, batch_size: int):
#   #   return DataLoader(self.dataset, batch_size = batch_size, shuffle=True)